<a href="https://colab.research.google.com/github/ridhyaathuppad/Exception-Handlers-ClauseWise/blob/main/ClauseWisecopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate torch gradio PyPDF2 python-docx pillow spacy --quiet
!python -m spacy download en_core_web_sm --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr
import PyPDF2
import docx
import re
import spacy
from typing import Dict, List, Tuple
import io

In [ ]:
print("Loading models...")
model_name = "ibm-granite/granite-3.2-2b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.3
)
nlp = spacy.load("en_core_web_sm")
print("✅ Models loaded successfully!")


Loading models...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cpu


✅ Models loaded successfully!


In [ ]:
def read_pdf(file_path) -> str:
    """Read PDF with enhanced error handling"""
    try:
        # Handle both file path string and file object
        if isinstance(file_path, str):
            with open(file_path, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                text = ""
                for page_num, page in enumerate(reader.pages):
                    try:
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text + "\n"
                    except Exception as e:
                        print(f"Warning: Could not read page {page_num + 1}: {e}")
                        continue
        else:
            # File object from Gradio
            reader = PyPDF2.PdfReader(file_path)
            text = ""
            for page_num, page in enumerate(reader.pages):
                try:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                except Exception as e:
                    print(f"Warning: Could not read page {page_num + 1}: {e}")
                    continue

        if text.strip():
            return text
        return "Error: PDF appears to be empty or image-based (scanned). Please use a text-based PDF."
    except Exception as e:
        return f"Error reading PDF: {str(e)}. Please ensure the file is not corrupted."

def read_docx(file_path) -> str:
    """Read DOCX files"""
    try:
        doc = docx.Document(file_path)
        text = "\n".join([p.text for p in doc.paragraphs if p.text.strip()])
        return text if text else "Error: Document appears to be empty"
    except Exception as e:
        return f"Error reading DOCX: {str(e)}"

def read_txt(file_path) -> str:
    """Read TXT files"""
    try:
        # Handle both file path and file object
        if isinstance(file_path, str):
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            content = file_path.read()
            if isinstance(content, bytes):
                return content.decode('utf-8', errors='ignore')
            return content
    except UnicodeDecodeError:
        # Fallback to latin-1 encoding
        try:
            if isinstance(file_path, str):
                with open(file_path, 'r', encoding='latin-1') as f:
                    return f.read()
            else:
                return file_path.read().decode('latin-1')
        except Exception as e:
            return f"Error reading TXT: {str(e)}"
    except Exception as e:
        return f"Error reading TXT: {str(e)}"

def read_file(file_path) -> str:
    """Universal file reader with comprehensive error handling"""
    if file_path is None:
        return ""

    try:
        # Get filename and extension
        if isinstance(file_path, str):
            filename = file_path
        else:
            filename = file_path.name if hasattr(file_path, 'name') else str(file_path)

        ext = filename.split(".")[-1].lower()

        print(f"Reading file: {filename} (type: {ext})")

        if ext == "pdf":
            result = read_pdf(file_path)
        elif ext == "docx":
            result = read_docx(file_path)
        elif ext == "txt":
            result = read_txt(file_path)
        else:
            return f"Unsupported file type: .{ext}. Please upload PDF, DOCX, or TXT files."

        # Log success
        if not result.startswith("Error"):
            print(f"✓ Successfully read {len(result)} characters from {filename}")

        return result

    except Exception as e:
        error_msg = f"Error processing file: {str(e)}"
        print(f"✗ {error_msg}")
        return error_msg

print("✅ Enhanced file readers initialized (PDF upload fixed!)")

✅ Enhanced file readers initialized (PDF upload fixed!)


In [ ]:
def simplify_clause(text: str) -> str:
    """Simplify complex legal language"""
    prompt = f"""Rewrite this legal text in simple, everyday language that anyone can understand:

Legal Text: {text[:1000]}

Simple Version:"""

    result = generator(prompt, max_new_tokens=300)[0]['generated_text']
    simplified = result.split("Simple Version:")[-1].strip()
    return simplified

print("✅ Simplification function ready")


✅ Simplification function ready


In [ ]:
def extract_entities(text: str) -> Dict[str, List[str]]:
    """Extract legal entities using spaCy + pattern matching"""
    doc = nlp(text)

    entities = {
        "Parties": [],
        "Dates": [],
        "Money": [],
        "Organizations": [],
        "Locations": [],
        "Legal Terms": [],
        "Obligations": []
    }

    # SpaCy NER
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            entities["Parties"].append(ent.text)
        elif ent.label_ == "DATE":
            entities["Dates"].append(ent.text)
        elif ent.label_ == "MONEY":
            entities["Money"].append(ent.text)
        elif ent.label_ == "ORG":
            entities["Organizations"].append(ent.text)
        elif ent.label_ in ["GPE", "LOC"]:
            entities["Locations"].append(ent.text)

    # Pattern matching for legal terms
    legal_patterns = [
        r'\b(agrees?|shall|must|may|will|warrants?|represents?|covenants?)\b',
        r'\b(liability|indemnif(?:y|ication)|breach|termination|confidential)\b',
        r'\b(jurisdiction|governing law|arbitration|dispute resolution)\b'
    ]

    for pattern in legal_patterns:
        matches = re.finditer(pattern, text, re.IGNORECASE)
        for match in matches:
            term = match.group(0)
            if term not in entities["Legal Terms"]:
                entities["Legal Terms"].append(term)

    # Extract obligations (sentences with "shall" or "must")
    obligation_pattern = r'[^.!?]*(?:shall|must)[^.!?]*[.!?]'
    obligations = re.findall(obligation_pattern, text, re.IGNORECASE)
    entities["Obligations"] = [o.strip() for o in obligations[:5]]

    # Remove duplicates
    for key in entities:
        entities[key] = list(set(entities[key]))[:10]

    return entities

def format_entities(entities: Dict) -> str:
    """Format entities for display"""
    output = "### 📋 NAMED ENTITY RECOGNITION\n\n"
    for category, items in entities.items():
        if items:
            output += f"**{category}:**\n"
            for item in items:
                output += f"  • {item}\n"
            output += "\n"
    return output

print("✅ NER functions ready")

✅ NER functions ready


In [ ]:
def extract_clauses(text: str) -> List[Tuple[str, str]]:
    """Extract and label individual clauses"""

    clause_patterns = [
        r'(?:^|\n)\s*(\d+\.?\d*\.?\s+[A-Z][^.\n]{10,80}\.)',
        r'(?:^|\n)\s*([A-Z][A-Z\s]{5,50}:)',
        r'(?:^|\n)\s*((?:WHEREAS|NOW THEREFORE|Article|Section|Clause)\s+[^\n]{10,80})',
    ]

    clauses = []
    sections = re.split(r'\n\s*\n', text)

    for i, section in enumerate(sections):
        if len(section.strip()) < 50:
            continue

        title = f"Clause {i+1}"
        for pattern in clause_patterns:
            match = re.search(pattern, section)
            if match:
                title = match.group(1).strip()
                break

        clauses.append((title, section.strip()[:500]))

    return clauses[:15]

def format_clauses(clauses: List[Tuple[str, str]]) -> str:
    """Format extracted clauses"""
    output = "### 📑 CLAUSE EXTRACTION\n\n"
    for i, (title, content) in enumerate(clauses, 1):
        output += f"**{i}. {title}**\n"
        output += f"{content[:300]}{'...' if len(content) > 300 else ''}\n\n"
    return output

print("✅ Clause extraction ready")


✅ Clause extraction ready


In [ ]:
def classify_document(text: str) -> str:
    """Classify legal document type"""

    text_lower = text.lower()
    keywords = {
        "NDA": ["non-disclosure", "confidential", "proprietary information"],
        "Lease Agreement": ["lease", "tenant", "landlord", "rent", "premises"],
        "Employment Contract": ["employment", "employee", "employer", "salary", "job duties"],
        "Service Agreement": ["services", "provider", "client", "deliverables"],
        "Partnership Agreement": ["partner", "partnership", "profit sharing"],
        "Purchase Agreement": ["purchase", "buyer", "seller", "goods"],
        "Loan Agreement": ["loan", "borrower", "lender", "interest rate"]
    }

    scores = {}
    for doc_type, terms in keywords.items():
        score = sum(1 for term in terms if term in text_lower)
        if score > 0:
            scores[doc_type] = score

    if scores:
        return max(scores, key=scores.get)

    prompt = f"""Classify this legal document into ONE category:
[NDA, Lease Agreement, Employment Contract, Service Agreement, Partnership Agreement, Purchase Agreement, Loan Agreement, Other]

Document excerpt: {text[:800]}

Classification:"""

    result = generator(prompt, max_new_tokens=20)[0]['generated_text']
    classification = result.split("Classification:")[-1].strip().split('\n')[0]
    return classification

print("✅ Document classifier ready")


✅ Document classifier ready


In [ ]:
# ========================================
# CELL 9: Main Processor with Multilingual
# ========================================

def process_document(file, text_input, operations, translate_option="No"):
    """Main processor with multilingual support"""

    # Read input
    if file is not None:
        text = read_file(file)
        if text.startswith("Error"):
            return f"❌ {text}\n\n**Troubleshooting:**\n• PDF must be text-based\n• Check file integrity\n• Try different format"
    elif text_input.strip():
        text = text_input
    else:
        return "⚠️ Please upload a file or paste text to analyze."

    if len(text.strip()) < 100:
        return "⚠️ Document too short (min 100 characters)."

    # Language detection
    detected_lang = detect_language_advanced(text)

    # Store original for display
    original_text = text

    # Translate if needed
    if detected_lang != "English" and translate_option == "Yes":
        text = translate_to_english(text, detected_lang)
        translation_note = f"\n> 🌍 **Translated from {detected_lang} to English for analysis**\n\n"
    else:
        translation_note = ""

    # Start report
    output = f"# 📄 ClauseWise Analysis Report\n\n"
    output += f"**Document Length:** {len(original_text):,} characters\n"
    output += f"**Detected Language:** {detected_lang}\n"
    output += translation_note
    output += "---\n\n"

    # Document classification
    doc_type = classify_document(text)
    output += f"### 📂 DOCUMENT TYPE\n**{doc_type}**\n\n---\n\n"

    # Named Entity Recognition
    if "Named Entity Recognition" in operations:
        entities = extract_entities(text)
        output += format_entities(entities)
        output += "---\n\n"

    # Clause Extraction
    if "Clause Extraction" in operations:
        clauses = extract_clauses(text)
        output += format_clauses(clauses)
        output += "---\n\n"

    # Simplification
    if "Clause Simplification" in operations:
        output += "### 💡 PLAIN-LANGUAGE SUMMARY\n\n"
        simplified = simplify_clause(text[:1500])
        output += f"{simplified}\n\n---\n\n"

    # Legal summary
    if "Legal Summary" in operations:
        output += "### ⚖️ LEGAL SUMMARY\n\n"
        prompt = f"""Provide a concise legal summary:
1. Purpose
2. Key obligations
3. Important terms

Document: {text[:1500]}

Summary:"""
        result = generator(prompt, max_new_tokens=400)[0]['generated_text']
        summary = result.split("Summary:")[-1].strip()
        output += f"{summary}\n\n"

    return output

print("✅ Main processor ready with multilingual support")

✅ Main processor ready with multilingual support


In [ ]:
# Quick diagnostic
print("Checking setup...")
print(f"✓ Model loaded: {model is not None}")
print(f"✓ Generator ready: {generator is not None}")
print(f"✓ spaCy loaded: {nlp is not None}")
print("Ready to launch Gradio!")

Checking setup...
✓ Model loaded: True
✓ Generator ready: True
✓ spaCy loaded: True
Ready to launch Gradio!


In [ ]:
# ========================================
# CELL 9B: Export & Multilingual Features
# ========================================

from datetime import datetime

# --- MULTILINGUAL SUPPORT ---
def translate_to_english(text: str, source_lang: str) -> str:
    """Translate non-English text to English for processing"""
    if source_lang == "English":
        return text

    prompt = f"""Translate this {source_lang} legal document to English. Preserve legal terminology and structure.

{source_lang} Text: {text[:1500]}

English Translation:"""

    try:
        result = generator(prompt, max_new_tokens=800)[0]['generated_text']
        translation = result.split("English Translation:")[-1].strip()
        return translation if translation else text
    except:
        return text

def detect_language_advanced(text: str) -> str:
    """Enhanced language detection"""
    text_sample = text[:500].lower()

    # Language indicators
    patterns = {
        "Spanish": ['artículo', 'contrato', 'cláusula', 'según', 'empresa', 'mediante'],
        "French": ['article', 'contrat', 'clause', 'selon', 'entreprise', 'conformément'],
        "German": ['artikel', 'vertrag', 'klausel', 'gemäß', 'unternehmen', 'vereinbarung'],
        "Hindi": ['अनुबंध', 'धारा', 'समझौता', 'के अनुसार'],
        "Portuguese": ['artigo', 'contrato', 'cláusula', 'conforme', 'empresa'],
        "Italian": ['articolo', 'contratto', 'clausola', 'secondo', 'impresa']
    }

    scores = {}
    for lang, keywords in patterns.items():
        score = sum(1 for keyword in keywords if keyword in text_sample)
        if score > 0:
            scores[lang] = score

    if scores:
        detected = max(scores, key=scores.get)
        return detected

    return "English"

# --- EXPORT TO PDF (Text-based) ---
def export_to_markdown(analysis_text: str, doc_type: str = "Legal Document") -> str:
    """Export analysis as downloadable markdown file"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"ClauseWise_Analysis_{timestamp}.md"

    try:
        # Create markdown content
        content = f"""# ClauseWise Analysis Report
**Generated:** {datetime.now().strftime("%B %d, %Y at %I:%M %p")}
**Document Type:** {doc_type}

---

{analysis_text}

---

*Report generated by ClauseWise AI - Legal Document Intelligence Platform*
*⚠️ This analysis is for informational purposes only and does not constitute legal advice.*
"""

        # Save to file
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)

        return filename
    except Exception as e:
        return f"Export failed: {str(e)}"

# --- EXPORT TO TXT ---
def export_to_txt(analysis_text: str) -> str:
    """Export as plain text"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"ClauseWise_Analysis_{timestamp}.txt"

    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"CLAUSEWISE AI ANALYSIS REPORT\n")
            f.write(f"Generated: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}\n")
            f.write("="*70 + "\n\n")
            # Remove markdown formatting for plain text
            clean_text = analysis_text.replace('#', '').replace('**', '').replace('*', '')
            f.write(clean_text)
            f.write("\n\n" + "="*70)
            f.write("\nReport by ClauseWise AI | For informational purposes only")

        return filename
    except Exception as e:
        return f"Export failed: {str(e)}"

print("✅ Export & Multilingual features loaded")

✅ Export & Multilingual features loaded


In [ ]:
# ========================================
# CELL 10: UI with Export & Multilingual
# ========================================

with gr.Blocks(css="""
    .gradio-container {font-family: 'Inter', sans-serif !important;}
    .header-section {
        background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
        padding: 2.5rem; border-radius: 12px; margin-bottom: 2rem;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }
    .header-title {color: white; font-size: 2.2rem; font-weight: 700;
        margin: 0; text-align: center;}
    .btn-primary {background: linear-gradient(135deg, #2a5298 0%, #1e3c72 100%) !important;
        border: none !important; font-weight: 600 !important; padding: 0.8rem 2rem !important;}
    .export-btn {background: #28a745 !important; color: white !important;}
""", title="ClauseWise Pro") as demo:

    gr.HTML("""
        <div class="header-section">
            <h1 class="header-title">⚖️ ClauseWise Pro</h1>
            <p style='color: white; text-align: center; margin-top: 0.5rem;'>
                AI-Powered Legal Intelligence with Export & Multilingual Support
            </p>
        </div>
    """)

    with gr.Row():
        with gr.Column(scale=2):
            gr.HTML("<h3 style='color: #1e3c72;'>📁 Upload Document</h3>")
            file_input = gr.File(
                label="",
                file_types=[".pdf", ".docx", ".txt"],
                type="filepath"
            )

            gr.HTML("<h3 style='color: #1e3c72; margin-top: 1rem;'>📝 Or Paste Text</h3>")
            text_input = gr.Textbox(label="", placeholder="Paste document text...", lines=6)

            gr.HTML("<h3 style='color: #1e3c72; margin-top: 1rem;'>🌍 Multilingual Options</h3>")
            translate_toggle = gr.Radio(
                ["No", "Yes"],
                label="Auto-translate to English?",
                value="No",
                info="Enable for Spanish, French, German, Hindi, Portuguese, Italian documents"
            )

            gr.HTML("<h3 style='color: #1e3c72; margin-top: 1rem;'>🔍 Analysis Options</h3>")
            operations = gr.CheckboxGroup(
                ["Named Entity Recognition", "Clause Extraction", "Clause Simplification", "Legal Summary"],
                label="",
                value=["Named Entity Recognition", "Clause Extraction"]
            )

            with gr.Row():
                analyze_btn = gr.Button("🚀 Analyze", variant="primary", size="lg")
                clear_btn = gr.ClearButton([file_input, text_input], value="🔄 Clear", size="lg")

        with gr.Column(scale=3):
            gr.HTML("<h3 style='color: #1e3c72;'>📊 Analysis Results</h3>")
            output_text = gr.Markdown(
                value="<div style='text-align:center; padding:3rem; color:#6c757d;'><div style='font-size:3rem;'>📄</div><h3>Ready to Analyze</h3><p>Upload document or paste text</p></div>"
            )

            gr.HTML("<h3 style='color: #1e3c72; margin-top: 1.5rem;'>💾 Export Options</h3>")
            with gr.Row():
                export_md_btn = gr.Button("📥 Export as Markdown", elem_classes=["export-btn"])
                export_txt_btn = gr.Button("📥 Export as Text", elem_classes=["export-btn"])

            export_status = gr.Textbox(label="Export Status", visible=False)
            download_file = gr.File(label="Download Your Report", visible=False)

    gr.HTML("""
        <div style='background: #f8f9fa; padding: 1.5rem; border-radius: 8px; margin-top: 2rem;'>
            <p style='color: #1e3c72; font-weight: 600; margin: 0;'>
                🌍 Supported Languages: English • Spanish • French • German • Hindi • Portuguese • Italian
            </p>
        </div>
    """)

    # Store analysis result
    analysis_result = gr.State()
    doc_type_state = gr.State()

    def analyze_and_store(file, text, ops, translate):
        result = process_document(file, text, ops, translate)
        doc_type = classify_document(text if text else read_file(file))
        return result, result, doc_type

    def export_markdown(analysis, doc_type):
        if not analysis or analysis.startswith("<div"):
            return "⚠️ Please analyze a document first", None
        filename = export_to_markdown(analysis, doc_type)
        return f"✅ Exported: {filename}", filename

    def export_text(analysis):
        if not analysis or analysis.startswith("<div"):
            return "⚠️ Please analyze a document first", None
        filename = export_to_txt(analysis)
        return f"✅ Exported: {filename}", filename

    analyze_btn.click(
        fn=analyze_and_store,
        inputs=[file_input, text_input, operations, translate_toggle],
        outputs=[output_text, analysis_result, doc_type_state]
    )

    export_md_btn.click(
        fn=export_markdown,
        inputs=[analysis_result, doc_type_state],
        outputs=[export_status, download_file]
    ).then(
        lambda: (gr.update(visible=True), gr.update(visible=True)),
        outputs=[export_status, download_file]
    )

    export_txt_btn.click(
        fn=export_text,
        inputs=[analysis_result],
        outputs=[export_status, download_file]
    ).then(
        lambda: (gr.update(visible=True), gr.update(visible=True)),
        outputs=[export_status, download_file]
    )

demo.launch(share=True, debug=True)
print("🚀 ClauseWise Pro with Export & Multilingual launched!")

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f223d188254f58d23a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
